This is a starter notebook for the [Kitchenware classification](https://www.kaggle.com/competitions/kitchenware-classification) competition on Kaggle

To get started:

- Join the competition and accept rules
- Download your Kaggle credentials file
- If you're running in Saturn Cloud, configure your instance to have access to access the kaggle credentials

When this is done, we can download the data. We need to execute the following cell only once

In [1]:
!kaggle competitions download -c kitchenware-classification
!mkdir data
!unzip kitchenware-classification.zip -d data > /dev/null
!rm kitchenware-classification.zip

 99%|█████████████████████████████████████▊| 1.62G/1.63G [00:26<00:00, 84.9MB/s]
100%|██████████████████████████████████████| 1.63G/1.63G [00:26<00:00, 66.5MB/s]


In [10]:
!ls

data  README.md  Untitled.ipynb


Now let's train a baseline model

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

2022-12-08 07:32:19.113563: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


First, we will load the training dataframe and split it into train and validation

In [2]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [3]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

Now let's create image generators

In [4]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [6]:
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)
base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

base = base_model(inputs, training=False)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(6)(vectors)

model = keras.Model(inputs, outputs)

In [7]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator
)

Epoch 1/2


139/139 [==============================] - 74s 502ms/step - loss: 0.6582 - accuracy: 0.8277 - val_loss: 0.6834 - val_accuracy: 0.8417
Epoch 2/2
139/139 [==============================] - 62s 446ms/step - loss: 0.3727 - accuracy: 0.8981 - val_loss: 0.6555 - val_accuracy: 0.8570


Now let's use this model to predict the labels for test data

In [9]:
df_test = pd.read_csv('data/test.csv', dtype={'Id': str})
df_test['filename'] = 'data/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,data/images/0678.jpg
1,3962,data/images/3962.jpg
2,9271,data/images/9271.jpg
3,5133,data/images/5133.jpg
4,8842,data/images/8842.jpg


In [10]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

Found 3808 validated image filenames.


In [11]:
y_pred = model.predict(test_generator)

119/119 [==============================] - 47s 390ms/step


In [12]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [13]:
predictions = classes[y_pred.argmax(axis=1)]

Finally, we need to prepare the submission

In [14]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len('data/images/'):-4]
del df_submission['filename']

In [15]:
df_submission[['Id', 'label']].to_csv('submission.csv', index=False)

In [16]:
!kaggle competitions submit kitchenware-classification -f submission.csv -m 'validation: 0.8570'

100%|██████████████████████████████████████| 38.8k/38.8k [00:00<00:00, 46.4kB/s]
Successfully submitted to Kitchenware Classification